In [1]:
import torch
import clip
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

d:\Users\miret\anaconda3\envs\cv_aivic\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Hyperparameters

In [3]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DEVICE

'cuda'

### CLIP

In [ ]:
clip_model, clip_preprocess = clip.load("ViT-B/32", device="cuda", jit=False)
clip_model.eval()

def get_clip_features(image_path):
    image = Image.open(image_path)
    image_input = clip_preprocess(image).unsqueeze(0).to("cuda")
    with torch.no_grad():
        image_features = clip_model.encode_image(image_input)
    return image_features

def get_clip_text_features(text):
    text_input = clip.tokenize([text]).to("cuda")
    with torch.no_grad():
        text_features = clip_model.encode_text(text_input)
    return text_features

def get_clip_similarity(image_features, text_features):
    image_features /= image_features.norm(dim=-1, keepdim=True)
    text_features /= text_features.norm(dim=-1, keepdim=True)
    similarity = (100.0 * image_features @ text_features.T).softmax(dim=-1)
    return similarity.item()